In [1]:
import pandas as pd
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.data import Sentence
import numpy as np

from sklearn.model_selection import train_test_split

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/tess/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load data

In [19]:
df = pd.read_csv("outputs/gold_aspects_data.csv")

In [20]:
df.head(30)

,Unnamed: 0,words,sentence,sent_id,labels_y
0,0,'','' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
1,0,CHAPTER,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
2,0,III,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
3,0,THE,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
4,0,CUMÆAN,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,B-aspect
5,0,SIBYL,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,B-aspect
6,0,A,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
7,0,part,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
8,0,of,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,O
9,0,the,'' CHAPTER III THE CUMÆAN SIBYL A part of the ...,0,B-aspect


In [8]:
#df.drop(columns=["Caption", "Filename"], inplace=True)

In [4]:
df.rename(columns={"Aspect": "text", "labels_y": "bio"}, inplace=True)

# Make 5 train-test split files

In [6]:
from sklearn.model_selection import KFold

In [7]:
kf = KFold(n_splits= 5, shuffle = False)

In [8]:
splits = []
for split in kf.split(df):
    splits.append(split[0])

In [67]:
def to_file(filename, sents_df):
    with open(f"/home/tess/experiments/train_test_splits/{filename}.txt", "w") as f:
        for el in sents_df: #for every sentence
            for text, label in zip(el[1]["text"], el[1]["bio"]):
                print("{}\t{}".format(text, label), file=f)
            print("\n", file = f)

In [68]:
for num, split in enumerate(kf.split(df)):
    num += 1
    
    train = df.iloc[split[0]]
    train_filename = "train_" + str(num) #make filename for train split

    train_sentences = train.groupby("Filename")
    to_file(train_filename, train_sentences)
    
    test = df.iloc[split[1]]
    test_filename = "test_" + str(num)
    
    test_sentences = test.groupby("Filename")
    to_file(test_filename, test_sentences)

# Train sequence classifier 
* BERT
* MACBERTH

In [2]:
data_folder = '/home/tess/experiments/train_test_splits/'

In [3]:
columns = {0: "text", 1: "bio" }

In [16]:
# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train_5.txt',
                              test_file='test_5.txt')

2023-12-08 15:13:05,857 Reading data from /home/tess/experiments/train_test_splits
2023-12-08 15:13:05,859 Train: /home/tess/experiments/train_test_splits/train_5.txt
2023-12-08 15:13:05,860 Dev: None
2023-12-08 15:13:05,861 Test: /home/tess/experiments/train_test_splits/test_5.txt
2023-12-08 15:13:07,564 No dev split found. Using 0% (i.e. 174 samples) of the train split as dev data


In [17]:
len(corpus.train) #sentences

1565

In [18]:
# 2. what label do we want to predict?
label_type = 'bio'

# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type, add_unk=False)
print(label_dict)

# 4. initialize fine-tuneable transformer embeddings WITH document context
embeddings = TransformerWordEmbeddings(model='emanjavacas/MacBERTh',
                                       layers="-1", #ONLY USE THE LAST LAYER (embeddings)
                                       subtoken_pooling="first",
                                       fine_tune=True, #adapt model to data
                                       use_context=True, #document context is considered during the embedding process (surrounding words, ...)
                                       )

# 5. initialize bare-bones sequence tagger (no CRF, no RNN, no reprojection)
# EMBEDDINGS ARE DIRECTLY USED without any linear projection
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type='bio',
                        use_crf=False,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. run fine-tuning
trainer.fine_tune('resources/taggers/sota-ner-flert',
                  learning_rate=5.0e-6,
                  mini_batch_size=4,
                  mini_batch_chunk_size=1,  # remove this parameter to speed up computation if you have a big GPU
                  )

2023-12-08 15:13:08,940 Computing label dictionary. Progress:


0it [00:00, ?it/s]
1565it [00:00, 22372.56it/s]

2023-12-08 15:13:09,019 Dictionary created for label 'bio' with 1 values: aspect (seen 5565 times)
Dictionary with 1 tags: aspect


2023-12-08 15:13:12,488 SequenceTagger predicts: Dictionary with 5 tags: O, S-aspect, B-aspect, E-aspect, I-aspect
2023-12-08 15:13:12,496 ----------------------------------------------------------------------------------------------------
2023-12-08 15:13:12,498 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30001, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True

100%|██████████| 11/11 [00:01<00:00,  7.39it/s]

...

100%|██████████| 11/11 [00:01<00:00,  5.65it/s]


2023-12-08 15:25:42,231 DEV : loss 0.3353153169155121 - f1-score (micro avg)  0.6152
2023-12-08 15:25:43,204 ----------------------------------------------------------------------------------------------------
2023-12-08 15:25:43,208 Testing using last state of model ...


100%|██████████| 28/28 [00:04<00:00,  6.63it/s]

2023-12-08 15:25:47,480 
Results:
- F-score (micro) 0.6099
- F-score (macro) 0.6099
- Accuracy 0.4388

By class:
              precision    recall  f1-score   support

      aspect     0.6442    0.5791    0.6099      1466

   micro avg     0.6442    0.5791    0.6099      1466
   macro avg     0.6442    0.5791    0.6099      1466
weighted avg     0.6442    0.5791    0.6099      1466

2023-12-08 15:25:47,482 ----------------------------------------------------------------------------------------------------


{'test_score': 0.6099137931034482}

In [22]:
# make a sentence
sentence = Sentence('I saw a weeping willow while passing down the alleyway. The thorns of the roses reminded me of home, in Alabama.')

# predict aspect tags
tagger.predict(sentence)

In [23]:
print(sentence)

Sentence[24]: "I saw a weeping willow while passing down the alleyway. The thorns of the roses reminded me of home, in Alabama." → ["willow"/aspect, "roses"/aspect, "Alabama"/aspect]
